In [1]:
using GradientRobustMultiPhysics
using ExtendableGrids
using Printf
using ExtendableSparse
using DelimitedFiles
using SimplexGridFactory
using Triangulate

In [2]:
"""
    get_flow_data(ν, example)

TBW
"""
function get_flow_data(ν, example)
    ## note that dependencies "XT" marks the function to be x- and t-dependent
    ## that causes the solver to automatically reassemble associated operators in each time step
    u = DataFunction((result, x, t) -> (
        result[1] = 3*t*(x[1]+x[2]); 
        result[2] = -3*t*(x[1]+x[2]);
        ), [2,2]; name = "u", dependencies = "XT", bonus_quadorder = 5)
    p = DataFunction((result, x) -> (
        result[1] = 0.0
        ), [1,2]; name = "p", dependencies = "X", bonus_quadorder = 5)
  
    ############## common code for all examples #####
    dt_u = eval_dt(u)
    Δu = eval_Δ(u)
    ∇u = eval_∇(u)
    ∇p = eval_∇(p)
    f = DataFunction((result, x, t) -> (
          result .= dt_u(x,t) .- ν*Δu(x,t) .+∇u(x,t)*u(x,t) .+ view(∇p(x,t),:);
        ), [2,2]; name = "f", dependencies = "XT", bonus_quadorder = 5)  
    return u, p, f
end

get_flow_data

In [3]:
function run_main(;scheme = 1, 
    ν = 1, 
    nlevels=3, 
    fe_type = 1, rec_type = 1, reconstruct = false,
    newton = true,
    T0 = 0, Tf=1, 
    nsteps=10, example = 1, verbosity = 0)

    # set log level
  set_verbosity(verbosity)
  ## initial grid 
  #xgrid = grid_unitsquare(Triangle2D)
  # refine the grid 
  #xgrid = uniform_refine(xgrid, nlevels)  
  xgrid = simplexgrid(Triangulate;
            points=[0 0 ; 0 1 ; 1 1 ; 1 0]',
            bfaces=[1 2 ; 2 3 ; 3 4 ; 4 1 ]',
            bfaceregions=[1, 2, 3, 4],
            regionpoints=[0.5 0.5;]',
            regionnumbers=[1],
            regionvolumes=[4.0^(-nlevels-1)/2])
  println(xgrid)

  ## choose one of these (inf-sup stable) finite element type pairs
  FETypes = [H1P2{2,2}, H1P1{1}] # Taylor-Hood elements
  
  u, p, f= get_flow_data(ν, example)

  # generate FE spaces
  FES = [FESpace{FETypes[1]}(xgrid), FESpace{FETypes[2]}(xgrid)]
  # solution vector
  Solution = FEVector(["u_h", "p_h"], FES)
  res = FEVector(["u_h", "p_h"], FES)
  Solnm1 = FEVector(FES)
  interpolate!(Solnm1[1], u; time = 0.0)
  Solnm2 = FEVector(FES)
  interpolate!(Solnm2[1], u; time = 1e-15)

  ndofu = FES[1].ndofs
  ndofp = FES[2].ndofs
  # n_unknown = length(Solution.entries)  
  GradientRobustMultiPhysics.interpolate!(Solution[1], u; time = 0.)

  # mass matrix 
  M = FEMatrix{Float64}(FES)
  assemble_operator!(M[1,1], BilinearForm([Identity, Identity]))
  # velocity pressure matrices
  A = FEMatrix{Float64}(FES)
  assemble_operator!(A[1,1], LaplaceOperator(ν))
  assemble_operator!(A[1,2], LagrangeMultiplier(Divergence); At = A[2,1]) 

  rhs = FEVector{Float64}(FES)
  assemble_operator!(rhs[1], LinearForm(Identity, f); time=0.0)

  ## the convection operator is assembled to the right-hand side
  ## to keep the matrix constant in time (but we do subiterations in each timestep)
  if (newton)
    function convection_kernel(result, input)
        uh, ∇uh = view(input,1:2), view(input,3:6)
        result[1] = ∇uh[1]*uh[1] + ∇uh[2]*uh[2]
        result[2] = ∇uh[3]*uh[1] + ∇uh[4]*uh[2]
    end
    CO = NonlinearForm(Identity, [Identity, Gradient], [1,1], convection_kernel, [2,6]; name = "((#1⋅∇)#1, #T)")
    DCO = GradientRobustMultiPhysics.AssemblyPattern{GradientRobustMultiPhysics.APT_NonlinearForm, Float64, ON_CELLS}(CO.name, 
        [FES[1], FES[1], FES[1]], CO.operators4arguments,CO.action,CO.apply_action_to,CO.regions)
    DCO.newton_args = CO.newton_arguments  
    @show DCO.newton_args
  else
    # disctrete convection operator
    # Picard iteration
    CO = ConvectionOperator(1, Identity, 2, 2; newton = newton)
    DCO = GradientRobustMultiPhysics.AssemblyPattern{GradientRobustMultiPhysics.APT_BilinearForm, Float64, ON_CELLS}(CO.name, 
    [FES[1], FES[1], FES[1]], CO.operators4arguments,CO.action,CO.apply_action_to,CO.regions)
  end
  
  # assemble the nonlinear matrix and rhs
  ANL = FEMatrix{Float64}(FES)  
  rhsNL = FEVector{Float64}(FES)
  # GradientRobustMultiPhysics.full_assemble!(ANL[1,1], rhsNL[1], DCO, [Solution[1], Solution[1]])
  dt = Array{BoundaryData,1}(undef,0)
  push!(dt, BoundaryData(BestapproxDirichletBoundary; regions = [1,2,3,4], data = u))
  dofs = boundarydata!(Solution[1], dt; time = 0.0)
  
  t0 = T0
  if scheme == 1
      tau = (Tf - T0)/nsteps
  end
  if scheme == 2
      tau = (Tf - T0)/nsteps
  end

  V1 = zeros(Float64, ndofu, 1)

  SystemMatrix = FEMatrix(FES)
  # @show SystemMatrix
  SystemRHS = FEVector(FES)
  SystemSol = FEVector(FES)
  
  step = one(Int)

  l2max = -one(Float64)
  first_step  = true
#------------------------------------------------------------------------------
  nts = 1
  oldL2 = zero(Float64); oldh1 = zero(Float64)
  eL2 = zero(Float64); eh1 = zero(Float64)
  tol = 1e-12
  max_iter = 10
  niter = zero(Int64)
  time_counter = 1
  @printf("\n\t STEP  |    TIME    |   RESIDUAL | NL-Ite |\n")
  while t0 <= Tf-1e-10
    t0 = t0 + tau    
    fill!(rhs.entries, 0)
    assemble_operator!(rhs[1], LinearForm(Identity, f), time= t0 )
    V1[:, 1] = rhs[1][:]

    fill!(SystemRHS.entries, 0)
    fill!(SystemMatrix.entries.cscmatrix.nzval, 0)
    fill!(A[1,1], 0)
    if nts == 1
      if scheme == 2
        @info ("First step is with BDF1")
      end
      addblock!(SystemRHS[1], M.entries*Solution[1].entries ; factor= 1.0/tau)
      addblock!(SystemRHS[1], V1[:,1]; factor= 1.0 )

      #system matrix
      assemble_operator!(A[1, 1], LaplaceOperator(ν); time=t0 )      
      addblock!(SystemMatrix[1, 1], M[1, 1]; factor= 1.0/tau)      
    else
      # system rhs
      addblock!(SystemRHS[1], M.entries*Solnm1[1].entries; factor=  2.0/tau)
      addblock!(SystemRHS[1], M.entries*Solnm2[1].entries; factor= -0.5/tau)
      addblock!(SystemRHS[1], V1[:,1]; factor= 1.0 )
      # system matrix 
      assemble_operator!(A[1, 1], LaplaceOperator(ν); time=t0 )
      addblock!(SystemMatrix[1, 1], M[1, 1]; factor= 1.5/tau)
    end
    addblock!(SystemMatrix[1, 1], A[1, 1]; factor= 1.0)
    addblock!(SystemMatrix[1, 2], A[1, 2]; factor= 1.0)
    addblock!(SystemMatrix[2, 1], A[2, 1]; factor= 1.0)

    niter = 0
    residual = 1e60
    flush!(SystemMatrix.entries)
    dofs = boundarydata!(Solution[1], dt; time = t0)
    
    while niter < max_iter && residual >= tol
      fill!(ANL.entries.cscmatrix.nzval, 0)
      fill!(rhsNL.entries, 0)
      flush!(ANL.entries)
      if newton  
        GradientRobustMultiPhysics.full_assemble!(ANL[1,1], rhsNL[1], DCO, [Solution[1], Solution[1]]; 
          skip_preps = false, factor = CO.factor, transposed_assembly = CO.transposed_assembly)        
      else
        # Assemble only ANL for Picard iteration 
        GradientRobustMultiPhysics.assemble!(ANL[1,1], DCO, [Solution[1], Solution[1]])
      end

      ## add linear part
      addblock!(rhsNL[1], SystemRHS[1]; factor= 1.0 )  
      addblock!(ANL[1, 1], SystemMatrix[1, 1]; factor= 1.0)
      addblock!(ANL[1, 2], SystemMatrix[1, 2]; factor= 1.0)
      addblock!(ANL[2, 1], SystemMatrix[2, 1]; factor= 1.0)  
      flush!(ANL.entries)
      
      for dof in dofs
        rhsNL[1][dof] = 1e60 * Solution[1][dof]
        ANL[1,1][dof,dof] = 1e60
      end
      # ANL[1,1][1,1] = 1e60
      ANL[2,2][1,1] = 1e60
      # residual computation
      GradientRobustMultiPhysics.mul!(res.entries,ANL.entries,Solution.entries)
      res.entries .-= rhsNL.entries
      for dof in dofs 
        res[1][dof] = 0
      end
      res[2][1] = 0
      residual = norm(res.entries)
      flush!(ANL.entries)
      # solve the system 
      Solution.entries[:] = ANL.entries \ rhsNL.entries

      niter = niter + 1
    end
    @printf("\t  %4d | %.4e | %.4e | %6d |\n", time_counter, t0, residual, niter)
    
    L2Error_u = L2ErrorIntegrator(u, Identity; time= t0)
    l2 = evaluate(L2Error_u, Solution[1])
    eL2 = (eL2 + (l2 + oldL2) )* tau * 0.5

    h1_err = L2ErrorIntegrator(∇(u), Gradient; time= t0)
    h1 = evaluate(h1_err, Solution[1])
    eh1 = (eh1 + l2 + oldh1) * tau * 0.5
    if scheme == 2
      nts = nts + 1
      for j=1:ndofu + ndofp
        Solnm2[1][j] = Solnm1[1][j]
        Solnm1[1][j] = Solution[1][j]
      end
    end
    # println(t0, "\t", sqrt(l2), '\t', sqrt(h1))
    time_counter = time_counter + 1
  end
  @show sqrt(eL2), sqrt(eh1)
end

run_main (generic function with 1 method)

In [4]:
function conv_test(ns, td=1, tdorder=0, ν=1, nl=2, fe_t=1, rec=1, recons=false )
    for n in ns
      #run_main2(nsteps = n)
      run_main(scheme=2, nsteps = n, newton=true)
    end
end
conv_test([10])

ExtendableGrid{Float64, Int32};
dim: 2 nodes: 443 cells: 808 bfaces: 76


┌ Warning:  39184772 allocations during ITEMTYPE_FACE volume calculation
└ @ ExtendableGrids ~/.julia/packages/ExtendableGrids/XFxI3/src/derived.jl:937
DCO.newton_args = [1, 2]

	 STEP  |    TIME    |   RESIDUAL | NL-Ite |
[ Info: First step is with BDF1
	     1 | 1.0000e-01 | 1.5430e-14 |      4 |
	     2 | 2.0000e-01 | 2.9611e-14 |      4 |
	     3 | 3.0000e-01 | 4.4251e-14 |      4 |
	     4 | 4.0000e-01 | 5.8406e-14 |      4 |
	     5 | 5.0000e-01 | 7.4203e-14 |      4 |
	     6 | 6.0000e-01 | 8.8704e-14 |      4 |
	     7 | 7.0000e-01 | 1.0470e-13 |      4 |
	     8 | 8.0000e-01 | 1.1818e-13 |      4 |
	     9 | 9.0000e-01 | 1.3113e-13 |      4 |
	    10 | 1.0000e+00 | 1.5047e-13 |      4 |
(sqrt(eL2), sqrt(eh1)) = (8.578427328608551e-16, 8.578427328608551e-16)
